In [1]:
%load_ext autoreload
%autoreload 2

from src.rxn_ctr_mcs import *
from src.utils import load_json, rxn_entry_to_smarts
from src.pathway_utils import get_reverse_paths_to_starting, create_graph_from_pickaxe
from src.post_processing import *

from minedatabase.pickaxe import Pickaxe
from minedatabase.utils import get_compound_hash

from rdkit.Chem import AllChem

import matplotlib.pyplot as plt
import numpy as np
from collections import defaultdict
import pandas as pd
import csv

/home/stef/miniconda3/envs/mine/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at /home/stef/miniconda3/envs/mine/lib/python3.7/site-packages/rxnmapper/models/transformers/albert_heads_8_uspto_all_1310k were not used when initializing AlbertModel: ['predictions.LayerNorm.bias', 'predictions.dense.weight', 'predictions.decoder.bias', 'predictions.decoder.weight', 'predictions.LayerNorm.weight', 'predictions.dense.bias', 'predictions.bias']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expec

## Main

In [2]:
# Set params

expansion_dir = '../data/raw_expansions/'
fn = "succinate_to_mvacid_gen_4_tan_sample_n_samples_1000.pk" # Expansion file name
generations = 4

# Load results
pk = Pickaxe()
path = expansion_dir + fn
pk.load_pickled_pickaxe(path)

----------------------------------------
Intializing pickaxe object

Done intializing pickaxe object
----------------------------------------

Loading ../data/raw_expansions/succinate_to_mvacid_gen_4_tan_sample_n_samples_1000.pk pickled data.
Loaded 89758 compounds
Loaded 106853 reactions
Loaded 3604 operators
Loaded 1 targets
Took 6.577129364013672


In [3]:
# Create the initial graph

DG, rxn, edge = create_graph_from_pickaxe(pk, "Biology")
starting_nodes = []
bad_nodes = []
for n in DG.nodes():
    try:
        if DG.nodes()[n]["Type"] == "Starting Compound":
            starting_nodes.append(n)
    except:
        bad_nodes.append(n)

RDKit ERROR: [14:51:17] Can't kekulize mol.  Unkekulized atoms: 19 23 26 27 28 29 30 31 33 34
[14:51:17] Can't kekulize mol.  Unkekulized atoms: 19 23 26 27 28 29 30 31 33 34

RDKit ERROR: 
RDKit ERROR: [14:51:18] Can't kekulize mol.  Unkekulized atoms: 19 23 26 27 28 29 30 31 33 34
RDKit ERROR: 
[14:51:18] Can't kekulize mol.  Unkekulized atoms: 19 23 26 27 28 29 30 31 33 34

RDKit ERROR: [14:51:25] Can't kekulize mol.  Unkekulized atoms: 19 23 26 27 28 29 30 31 33 34
RDKit ERROR: 
[14:51:25] Can't kekulize mol.  Unkekulized atoms: 19 23 26 27 28 29 30 31 33 34

RDKit ERROR: [14:51:26] Can't kekulize mol.  Unkekulized atoms: 19 23 26 27 28 29 30 31 33 34
RDKit ERROR: 
[14:51:26] Can't kekulize mol.  Unkekulized atoms: 19 23 26 27 28 29 30 31 33 34

RDKit ERROR: [14:51:26] Can't kekulize mol.  Unkekulized atoms: 19 23 26 27 28 29 30 31 33 34
RDKit ERROR: 
[14:51:26] Can't kekulize mol.  Unkekulized atoms: 19 23 26 27 28 29 30 31 33 34

RDKit ERROR: [14:51:26] Can't kekulize mol.  Unkek

In [ ]:
# Get pathways
max_depth = generations * 2
paths = []

# Specify Targets / Starting Cpds
target_smi = pk.target_smiles[0]
target_cids = [get_compound_hash(smi)[0] for smi in pk.target_smiles]
starting_cpds = [get_compound_hash(val["SMILES"])[0] for val in pk.compounds.values() if val["Type"].startswith("Start")]

# Loop through targets and get pathways from targets to starting compounds
for this_target in target_cids:
    this_paths = get_reverse_paths_to_starting(DG, begin_node=this_target, end_nodes=starting_cpds, max_depth=max_depth)
    # If we find paths then reverse those paths and assign to a dictionary
    if this_paths:
        this_paths = list(set([tuple(path[1::2]) for path in [[*reversed(ind_path)] for ind_path in this_paths]]))
        for elt in this_paths:
            for r in pk.reactions[elt[0]]["Reactants"]:
                if r[-1] in starting_cpds:
                    paths.append(pathway(rhashes=elt, starter_hash=r[-1], target_hash=this_target)) 

In [ ]:
# Make predicted reaction dict

pred_rxns = {}
degen_rhashes = defaultdict(lambda : 1)
for elt in paths:
    for this_rhash in elt.rhashes:
        if this_rhash not in pred_rxns:
            rxn_sma = rxn_hash_2_rxn_sma(this_rhash, pk)
            pred_rxns[this_rhash] = reaction(this_rhash, rxn_sma)
        else:
            degen_rhashes[this_rhash] += 1


In [ ]:
# Put paths in dict indexed by starter-target pairs
# target_id_2_name = {'Cec77ea281f69ca989bbef0a4c7794128a196c716': '2-methyleneglutarate',
#                     'C3a5b833f1a5abe2063ff8b22706fabb74097bead': '4-methyleneglutamate'}
target_id_2_name = {'C6ec1611229ff4fc7a19244967c7716266fc021a1': 'mvacid'}

st_paths = defaultdict(list)
for elt in paths:
    t_name = target_id_2_name[elt.target]
    s_name = pk.compounds[elt.starter]["ID"]
    st_paths[(s_name, t_name)].append(elt)

for k,v in st_paths.items():
    print(k, len(v))

('succinate', 'mvacid') 41


In [ ]:
# Load in IMT rule mapping

# Load rules
rules_path = '../src/rules/JN3604IMT_rules.tsv'
rule_df = pd.read_csv(rules_path, delimiter='\t')
rule_df.set_index('Name', inplace=True)

# Load mapping
rxn2rule = {}
db_names = ['_mc_v21', '_brenda', '_kegg']
suffix = '_imt_rules_enforce_cof.csv'
for name in db_names:
    mapping_path = '../data/mapping/mapping' + name + suffix
    with open(mapping_path, 'r') as f:
        reader = csv.reader(f)
        for row in reader:
            if len(row) == 1:
                rxn2rule[row[0]] = []
            else:
                rxn2rule[row[0]] = row[1:]

# Make rule2rxn
rule2rxn = {}
for k,v in rxn2rule.items():
    for elt in v:
        if elt not in rule2rxn:
            rule2rxn[elt] = [k]
        else:
            rule2rxn[elt].append(k)

# Load all known reaction json entries into dict
known_rxns = {}
pref = '../data/mapping/'
suffs = ['mc_v21_as_is.json', 'brenda_as_is.json', 'kegg_as_is.json']
for elt in suffs:
    known_rxns.update(load_json(pref + elt))

In [ ]:
# Populate reaction objects in rxn dict w/ known reactions

for k, v in pred_rxns.items():
    this_rules = list(pk.reactions[k]["Operators"])
    this_known_rxns = []
    for elt in this_rules:
        if elt in rule2rxn:
            this_rxn_ids = rule2rxn[elt]
            for this_id in this_rxn_ids:
                this_sma = rxn_entry_to_smarts(known_rxns[this_id])
                this_known_rxns.append((None, this_sma, this_id))
    
    v.known_rxns = [list(elt) for elt in set(this_known_rxns)]


In [ ]:
# Populate pred_rxns, known rxn prc-mcs slot

for x in range(len(pred_rxns.keys())):
    h = list(pred_rxns.keys())[x]
    test_rxn = pred_rxns[h]
    a = 0 # Number known rxns analyszed
    for z, kr in enumerate(test_rxn.known_rxns):

        rxn_sma1 = test_rxn.smarts
        rxn_sma2 = kr[1]

        # Catch stoichiometry mismatches stemming from pickaxe, early post-processing
        if tuple([len(elt.split('.')) for elt in rxn_sma2.split('>>')]) != tuple([len(elt.split('.')) for elt in rxn_sma1.split('>>')]):
            print(x, z, 'stoich_error')
            continue

        # Skip pred reactions that trigger RXNMapper atom mapping errors
        try:
            am_rxn_smarts = [atom_map(rxn_sma1), atom_map(rxn_sma2)] # Atom map
        except:
            continue

        # Construct reaction objects
        rxns = []
        for elt in am_rxn_smarts:
            temp = AllChem.ReactionFromSmarts(elt, useSmiles=True)
            temp.Initialize()
            rxns.append(temp)

        rc_atoms = [elt.GetReactingAtoms() for elt in rxns] # Get reaction center atom idxs

        # Construct rxn ctr mol objs
        rcs = []
        for i, t_rxn in enumerate(rxns):
            temp = []
            for j, t_mol in enumerate(t_rxn.GetReactants()):
                temp.append(get_sub_mol(t_mol, rc_atoms[i][j]))
            rcs.append(temp)

        # Align substrates of the 2 reactions
        rc_idxs = [] # Each element: (idx for rxn 1, idx for rxn 2)
        remaining = [[i for i in range(len(elt))] for elt in rcs]
        while (len(remaining[0]) > 0) & (len(remaining[1]) > 0):
            idx_pair = align_substrates(rcs, remaining)

            if idx_pair is None:
                break
            else:
                rc_idxs.append(idx_pair)
                remaining[0].remove(idx_pair[0])
                remaining[1].remove(idx_pair[1])

        # Skip if you haven't aligned every reactant pred to known
        if len(rc_idxs) < len(rxn_sma1.split('>>')[0].split('.')):
            continue

        rxns = align_atom_map_nums(rxns, rcs, rc_idxs, rc_atoms)

        # Compute MCS seeded by reaction center
        prc_mcs = get_prc_mcs(rxns, rcs, rc_idxs, rc_atoms) 
        pred_rxns[h].known_rxns[z][0] = prc_mcs # Update pred_rxns
        
        a += 1

    print(x, ':', a / (z+1), 'out of', z+1)

Token indices sequence length is longer than the specified maximum sequence length for this model (703 > 512). Running this sequence through the model will result in indexing errors


0 : 0.15 out of 40
1 : 0.6086956521739131 out of 23
2 : 1.0 out of 24
3 : 1.0 out of 18
4 : 0.0 out of 9
5 : 0.13636363636363635 out of 88
6 : 1.0 out of 9
7 : 0.6086956521739131 out of 23
8 : 0.0 out of 88
9 : 0.0 out of 9
10 : 0.13636363636363635 out of 88
11 : 1.0 out of 18
12 : 0.723404255319149 out of 47
13 : 1.0 out of 20
14 : 0.0 out of 9
15 : 0.6086956521739131 out of 23
16 : 0.13636363636363635 out of 88
17 : 1.0 out of 24
18 : 0.9310344827586207 out of 29
19 : 1.0 out of 20
20 : 1.0 out of 9
21 : 0.15 out of 40
22 : 0.13636363636363635 out of 88
23 : 0.9310344827586207 out of 29
24 : 0.15 out of 40
25 : 0.723404255319149 out of 47
26 : 0.15 out of 40
27 : 0.0 out of 40
28 : 0.13636363636363635 out of 88
29 : 0.13636363636363635 out of 88
30 : 1.0 out of 18
31 : 0.15 out of 40
32 : 0.15 out of 40


In [ ]:
import pickle
rxns_fn = 'predicted_reactions_' + fn
save_dir = '../artifacts/'
path = save_dir + rxns_fn

with open(path, 'wb') as f:
    pickle.dump(pred_rxns, f)

with open(path, 'rb') as f:
    pred_rxns_loaded = pickle.load(f)

In [ ]:
# Count how many pathways have full info

n_paths_w_full_info = 0
for p in paths:
    path_full_info = []
    for rh in p.rhashes:
        full_info = False
        for elt in pred_rxns_loaded[rh].known_rxns:
            if elt[0] is not None:
                full_info = True
                break

        path_full_info.append(full_info)
    
    if all(path_full_info):
        n_paths_w_full_info += 1

print(n_paths_w_full_info / len(paths), len(paths))


0.5365853658536586 41


In [ ]:
# Set prc mcs value for path to average over all reactions
for p in paths:
    for rh in p.rhashes:
        substrate_mcs = []
        for kr in pred_rxns_loaded[rh].known_rxns:
            substrate_mcs.append()

TypeError: append() takes exactly one argument (0 given)

In [ ]:
# pred_known_pairs = []
# example_path = st_paths[('2-methyleneglutarate', 'mvacid')][3].rhashes
# rng = np.random.default_rng(seed=1208)
# for elt in example_path:
#     this_pred = pred_rxns[elt]
#     this_idx = rng.integers(0, len(this_pred.known_rxns) - 1)
#     pred_img = draw_rxn(this_pred.smarts)
#     known_img = draw_rxn(this_pred.known_rxns[this_idx][1])
#     pred_known_pairs.append([pred_img, known_img])

# path_img = draw_pathway(pred_known_pairs)
# path_img.save('../artifacts/test_concat.png')
    